In [367]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import io

In [368]:
import mlflow
import mlflow.pytorch

In [369]:
mlflow.set_experiment("MLP_Clasificador_Imagenes")

<Experiment: artifact_location='file:///c:/Users/ConstanzaBaiardi/Documents/CursoITBA/skin-dataset-classification/mlruns/1', creation_time=1766094357481, experiment_id='1', last_update_time=1766094357481, lifecycle_stage='active', name='MLP_Clasificador_Imagenes', tags={}>

In [370]:
from torch.utils.tensorboard import SummaryWriter
import torchvision.utils as vutils

In [371]:
# Función para loguear una figura matplotlib en TensorBoard
def plot_to_tensorboard(fig, writer, tag, step): #agarra una figura de matplotlib y la pasa a tensorflow
    buf = io.BytesIO() #archivo temporal en memoria
    fig.savefig(buf, format='png')
    buf.seek(0)
    image = Image.open(buf).convert("RGB")
    image = np.array(image)
    image = torch.tensor(image).permute(2, 0, 1) / 255.0
    writer.add_image(tag, image, global_step=step) #en tensorboard vamos a loggear toda la dinamica del entrenamiento y despues lo exportamos a mlflow
    plt.close(fig)

In [372]:
# Función para matriz de confusión y clasificación
def log_classification_report(model, loader, writer, step, prefix="val"):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())

    # Confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    fig_cm, ax = plt.subplots(figsize=(6, 6))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=train_dataset.label_encoder.classes_)
    disp.plot(ax=ax, cmap='Blues', xticks_rotation=45)
    ax.set_title(f'{prefix.title()} - Confusion Matrix')

    # Guardar localmente y subir a MLflow
    fig_path = f"confusion_matrix_{prefix}_epoch_{step}.png"
    fig_cm.savefig(fig_path)
    mlflow.log_artifact(fig_path)
    os.remove(fig_path)

    plot_to_tensorboard(fig_cm, writer, f"{prefix}/confusion_matrix", step)

    cls_report = classification_report(all_labels, all_preds, target_names=train_dataset.label_encoder.classes_)
    writer.add_text(f"{prefix}/classification_report", f"<pre>{cls_report}</pre>", step)

    # También loguear texto del reporte
    with open(f"classification_report_{prefix}_epoch_{step}.txt", "w") as f:
        f.write(cls_report)
    mlflow.log_artifact(f.name)
    os.remove(f.name)


In [373]:
# Crear directorio de logs
log_dir = "runs/mlp_experimento_1"
writer = SummaryWriter(log_dir=log_dir)


In [374]:
class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform

        self.image_paths = []
        self.labels = []

        class_names = sorted(os.listdir(root_dir))
        self.class_to_idx = {cls: idx for idx, cls in enumerate(class_names)}

        for cls in class_names:
            cls_dir = os.path.join(root_dir, cls) #recprrp todos los archivos del directorio y todos los que terminen en png,jpg y jpeg los agarro
            for fname in os.listdir(cls_dir):
                if fname.lower().endswith((".png", ".jpg", ".jpeg")):
                    self.image_paths.append(os.path.join(cls_dir, fname))
                    self.labels.append(cls)

        self.label_encoder = LabelEncoder()
        self.labels = self.label_encoder.fit_transform(self.labels)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx): #nunca entro con la imagen original
        image = np.array(Image.open(self.image_paths[idx]).convert("RGB"))
        label = self.labels[idx] #devuelvo la imagen y la etiqueta numerica entre 0 y 8

        if self.transform: #si se tanforma la imagen por algun motivo, dame la imagen aumentada
            augmented = self.transform(image=image)
            image = augmented["image"]

        return image, label

In [375]:
#usamos la librerira albumentations para hacer aumentacion de datos, tambien aumenta las anotaciones cuando corresponda
train_transform = A.Compose([
    A.Resize(64, 64), #super necesario para que todas las imagenes tengan el mismo tamaño
    
     #MODIFICACIONES PUNTO 4
      
    A.HorizontalFlip(p=0.5), #la mitad de las veces le doy vuelta horizontalmente la imagen
    A.RandomBrightnessContrast(p=0.2), #la mitad de las veces le cambio el brillo y el contraste
    A.ShiftScaleRotate(
       shift_limit=0.05,
       scale_limit=0.1,
       rotate_limit=15,
       p=0.5
    ),
    A.Normalize(),
    ToTensorV2() #para entrar al modelo tiene que estar en formato tensor
])


c:\Users\Constanza Baiardi\anaconda\envs\dl\lib\site-packages\albumentations\core\validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


In [376]:
val_test_transform = A.Compose([ 
    A.Resize(64, 64),
    A.Normalize(),
    ToTensorV2()
])

In [377]:
# Paths
train_dir = "data/Split_smol/train"
val_dir = "data/Split_smol/val/"

In [378]:
train_dataset = CustomImageDataset(train_dir, transform=train_transform)
val_dataset   = CustomImageDataset(val_dir, transform=val_test_transform)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size)

In [379]:
class MLPClassifier(nn.Module):
    def __init__(self, input_size=64*64*3, num_classes=10):
        super().__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.model(x)

In [380]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = len(set(train_dataset.labels))
model = MLPClassifier(num_classes=num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)

In [381]:
for name, param in model.named_parameters():
    if "weight" in name:
        print(name, param.mean().item(), param.std().item())

model.1.weight 4.776884452439845e-07 0.005208674818277359
model.3.weight -1.6282203432638198e-05 0.02549908496439457
model.5.weight -0.0004465052334126085 0.04967690631747246


In [382]:
# Entrenamiento y validación
def evaluate(model, loader, epoch=None, prefix="val"):
    log_classification_report(model, val_loader, writer, step=epoch, prefix="val")
    model.eval()
    correct, total, loss_sum = 0, 0, 0.0

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for i, (images, labels) in enumerate(loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)

            loss_sum += loss.item()
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            # Loguear imágenes del primer batch
            if i == 0 and epoch is not None:
                img_grid = vutils.make_grid(images[:8].cpu(), normalize=True)
                writer.add_image(f"{prefix}/images", img_grid, global_step=epoch)

    acc = 100.0 * correct / total
    avg_loss = loss_sum / len(loader)

    if epoch is not None:
        writer.add_scalar(f"{prefix}/loss", avg_loss, epoch)
        writer.add_scalar(f"{prefix}/accuracy", acc, epoch)

    return avg_loss, acc

In [383]:
# Loop de entrenamiento
n_epochs = 20
with mlflow.start_run():
    # Log hiperparámetros
    mlflow.log_params({
        "model": "MLPClassifier",
        "input_size": 64*64*3,
        "batch_size": batch_size,
        "lr": 1e-3,
        "epochs": n_epochs,
        "optimizer": "Adam",
        "loss_fn": "CrossEntropyLoss",
        "train_dir": train_dir,
        "val_dir": val_dir,
    })
    for epoch in range(n_epochs):
        model.train()
        running_loss = 0.0
        correct, total = 0, 0
    
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{n_epochs}"):
            images, labels = images.to(device), labels.to(device)
    
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    
            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        if epoch == 0:
            for name, param in model.named_parameters():
                writer.add_histogram(name, param, epoch)
        
        train_loss = running_loss / len(train_loader)
        train_acc = 100.0 * correct / total
        val_loss, val_acc = evaluate(model, val_loader, epoch=epoch, prefix="val")
    
        print(f"Epoch {epoch+1}:")
        print(f"  Train Loss: {train_loss:.4f}, Accuracy: {train_acc:.2f}%")
        print(f"  Val   Loss: {val_loss:.4f}, Accuracy: {val_acc:.2f}%")
    
        writer.add_scalar("train/loss", train_loss, epoch)
        writer.add_scalar("train/accuracy", train_acc, epoch)
    
        # Log en MLflow
        mlflow.log_metrics({
            "train_loss": train_loss,
            "train_accuracy": train_acc,
            "val_loss": val_loss,
            "val_accuracy": val_acc
        }, step=epoch)
        # Guardar modelo
    torch.save(model.state_dict(), "mlp_model.pth")
    print("Modelo guardado como 'mlp_model.pth'")
    mlflow.log_artifact("mlp_model.pth")
    mlflow.pytorch.log_model(model, name="pytorch_model")
    print("Modelo guardado como 'mlp_model.pth'")

Epoch 1/20: 100%|██████████| 22/22 [00:08<00:00,  2.74it/s]
c:\Users\Constanza Baiardi\anaconda\envs\dl\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\Constanza Baiardi\anaconda\envs\dl\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\Constanza Baiardi\anaconda\envs\dl\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control thi

Epoch 1:
  Train Loss: 2.4737, Accuracy: 27.69%
  Val   Loss: 1.7539, Accuracy: 40.33%


Epoch 2/20: 100%|██████████| 22/22 [00:06<00:00,  3.32it/s]
c:\Users\Constanza Baiardi\anaconda\envs\dl\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\Constanza Baiardi\anaconda\envs\dl\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\Constanza Baiardi\anaconda\envs\dl\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control thi

Epoch 2:
  Train Loss: 1.7944, Accuracy: 40.32%
  Val   Loss: 1.7649, Accuracy: 34.25%


Epoch 3/20: 100%|██████████| 22/22 [00:06<00:00,  3.28it/s]


...

Epoch 18:
  Train Loss: 1.0306, Accuracy: 56.67%
  Val   Loss: 1.1162, Accuracy: 58.01%


Epoch 19/20: 100%|██████████| 22/22 [00:06<00:00,  3.17it/s]


Epoch 19:
  Train Loss: 0.9850, Accuracy: 60.83%
  Val   Loss: 1.2159, Accuracy: 57.46%


Epoch 20/20: 100%|██████████| 22/22 [00:06<00:00,  3.20it/s]


Epoch 20:
  Train Loss: 0.9174, Accuracy: 63.56%
  Val   Loss: 1.2239, Accuracy: 55.80%
Modelo guardado como 'mlp_model.pth'
Modelo guardado como 'mlp_model.pth'


In [384]:
%load_ext tensorboard
%tensorboard --logdir=runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6022 (pid 2900), started 0:04:49 ago. (Use '!kill 2900' to kill it.)